In [ ]:
# grabbing dataset to train our own court keypoints detecting model

# Data returns: 1. images 2. data_train.json 3. data_val.json

# data_train.json contains datapoints on keypoints of the tennis court broken down into ID (name of the image itself) and KPS (key points, 14 points of (x, y) coordinates to mark each point on the court)


!wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8" --header="Cookie: HSID=Ag2OIHvsd2Wub4C7z; SSID=AWnBcQKwDHiTrZAU1; APISID=pltrFZgE9lJ0o1gq/AN9feEHYvs8oHd519; SAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-1PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; __Secure-3PAPISID=zgF45F21ZPWzYWZw/AgUMJ8b7QQXuWGn19; SID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-uttBbVDRolhF-hY16nwHXw0gACgYKAWISAQASFQHGX2MivNTw_E_toJuIRy6LMpKNOBoVAUF8yKpFSmvq7AMjvEWeNc50Zff40076; __Secure-1PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utbSY2jBY1VXuw8gYl5hIO2QACgYKAXsSAQASFQHGX2MihVCJ1PwLozGqZgdSatM9QhoVAUF8yKpgrsTvI8i_UE-YHpoN7Gx-0076; __Secure-3PSID=g.a000fwgYx1PcnW-rFyFhg3x6mQHzCrwXz-KFhoOLogUl7YTWI-utwVfPl2imdPimZJ9tdDZGQAACgYKAUESAQASFQHGX2MiEJ49mV4jME2kttDAV5hwWBoVAUF8yKp80mIgju1lu-q4nI7VsFDM0076; NID=511=efI9IZpxtyJ7Dw1MAUXU8FlzS5jXGewY4Er8HliWc3A0RSWdgvNDyKY66ETjgRyTGWPbWODSmiSeYSBab5SPHVwqbJxd6ZeGW2f6BkHi61UKksXPH0CVJRM1hKpMjHPU5qw7tboM2Mi87NrosV8COB-GCLulLLbjOoSAEQewTe8NVZ5Owq8IkwvxFGfJkmUKEMkFWrw9yb5nTDl3wbZEsGFI92iEdNTSxSRovNCIPN2US-SCFdQ0m2BtvwdiWZbgnn7dSQ8yPA145Kk2BA-ATpJNJ6SJHEHLQY-9CPail9D5qgJgxR925EUg5RGCpEu9wS5xbA62KTa19wAvbAq7Dk3TWc-iX4p1s7ESFyDC7yMpFxiFPJjqkWwFi_ZfiK2TW2t0TQ60DFBxqOytQaLyHrkEvD-CQPVj6OCOP22cZY0Cu61HaAQgFO9pXH-kJUlywzVdbirJumN5gswyaQ49b3KdLcG0Jb7brOMTM24T2nGtQ10hJzsnTwX7dBk3ujqQrI_DGuURvPassPUrIZ0; AEC=Ae3NU9MOEGeKAZjP6INpOYbyMraWAWztmx5pJB_1ILu1furiTy1K37k15u0; __Secure-1PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; __Secure-3PSIDTS=sidts-CjEBYfD7Z9twEKTWJ9gU7KG-rLbxJGNRQIoG3wH6JVu6yiCC2fsRrm7tN8L6d5WlILrnEAA; 1P_JAR=2024-02-18-08; SIDCC=ABTWhQExCxkfmwCkG1RaEgz8U1ZkPeh3HmLMUdMt8S5cNSsLY5U5rAL6wlvq7dtjRw7zrtAbqsFI; __Secure-1PSIDCC=ABTWhQH0jLeRIS6Tu3LS8DXB5Q3gGDq9LTmlk60FKu795Bf0UbzsOcYWVAE96clq5aAL8i724Q0; __Secure-3PSIDCC=ABTWhQHIFcyv3nZYwp78WXEQal71jCE_ZsGT5lXs8VLr7XDIfFqHcLTIPz4HxzJb9ZnYQ5l2s9eU" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040" -c -O 'tennis_court_det_dataset.zip'

In [ ]:
!unzip tennis_court_det_dataset.zip

# Training for marking keypoints on a tennis court 

# useful for marking tennis player travel distance and speed 


# Start code

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2 # used for image loading and processing
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [6]:
# pytorch dataset class
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        # establish image directory for training
        self.img_dir = img_dir
        # load the data file.json
        with open(data_file, "r") as f:
            self.data = json.load(f)

        # standarize and normalize and make all training data images the same
        self.transforms = transforms.Compose([
            # transform to PIL image type
            transforms.ToPILImage(),
            # resize all images to specific size
            transforms.Resize((224, 224)),

            transforms.ToTensor(),
            # normalization
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    # define the length of the dataset
    def __len__(self):
        return len(self.data)

    # Grabs a training image one at a time to train
    def __getitem__(self, idx):

        item = self.data[idx]

        # read in the image itself
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")

        # get the height and width of the image
        h,w = img.shape[:2]

        # cv2 reads image in BGR format, convert to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # apply the transformations to the image
        img = self.transforms(img)

        # get the keypoints and convert them to a numpy array so it can be used to train
        # flatten = convert N-D array to 1D
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # normalizing the x and y coordinates of the keypoints
        # multiple width by (normalized width / original width)
        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:

# create the training and validation datasets
# inputs to keypoints data set is the image input directory then the training .json file
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Create model

In [ ]:
# downloads and useds a pretrained model trained on other images
# foundation model
model = models.resnet50(pretrained=True)

# fine tuning section:
# modifies the last layer of the neural network to account for us trying to find tennis court key points
# model.fc = full connected layer = last layer
# 14 * 2 = 14 points on the court each with 2 coordinates (x, y) = change accourdingly based on other needs
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [ ]:
model = model.to(device)

# Train model

In [ ]:
# regression function problem

# define loss function
criterion = torch.nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
# epoch = the number of passings through a dataset
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        # flush out the gradients
        optimizer.zero_grad()
        outputs = model(imgs)

        # calculate the loss
        loss = criterion(outputs, kps)

        # backward propagation
        loss.backward()

        optimizer.step()

        # for logging
        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

In [ ]:
# saving the model loss points
torch.save(model.stat_dict(), "keypoints_model.pth")